<a href="https://colab.research.google.com/github/milagrosonofri/Problem_Set_1/blob/main/PS1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Obtención y limpieza de la base de datos

In [69]:
# Importamos las librerías necesarias
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

Web Scraping

In [ ]:
# Armamos una lista para almacenar todos los datos
data = []

# Base URL de las páginas
base_url = 'https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_{}.html'

# Iteramos sobre las 10 páginas
for i in range(1, 11):

    # Construimos la URL de la página actual
    url = base_url.format(i)

    # Enviamos solicitud HTTP
    response = requests.get(url)
    print(f"Scraping página {i}: {response}")  # Confirmar que la solicitud es exitosa (<Response [200]>)

    # Parsear el contenido HTML
    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontrar la tabla en la página
    table = soup.find('table')

    # Extraemos encabezados (solo en la primera iteración)
    if i == 1:  # Tomar encabezados solo de la primera página
        headers = []
        for header in table.find_all('th'):
            headers.append(header.text.strip())

    # Extraemos filas de datos
    for row in table.find_all('tr'):
        row_data = []
        for cell in row.find_all('td'):
            row_data.append(cell.text.strip())
        if row_data:  # Agregar solo filas con datos
            data.append(row_data)

# Convertimos los datos consolidados en un DataFrame
df = pd.DataFrame(data, columns=headers)

df

Scraping página 1: <Response [200]>
Scraping página 2: <Response [200]>


Limpieza de base de datos

In [ ]:
# Obtengo el nombre de todas las variables disponibles
variables = df.columns.tolist()

print(variables)

# Restringimos a individuos empleados mayores de 18 años (HAY QUE REVISAR SI P6240 ES LA CORRECTA)
df_limpio = df[(df['age'].astype(float) > 18) & (df['ocu'] == '1')]

print(f"Número de registros después del filtro (empleados mayores de 18 años): {len(df_limpio)}")

In [ ]:
# Primer approach para describir los datos

print("Información del DataFrame:")
print(df_limpio.info())

In [ ]:
# Renombro variables para tener una interpretación mas sencilla de aquí en adelante
df_limpio = df_limpio.rename(columns={'p6500': 'salario_empleo_principal', 'p7070': 'salario_empleo_secundario',
                                      'p6426': 'antig', 'sex': 'sexo', 'age': 'edad'})

In [ ]:
# Convierto algunas variables de relacionados al salario horario a formato numérico para poder compararlas
wage_hours_related_vars = ['salario_empleo_principal', 'salario_empleo_secundario', 'hoursWorkUsual', 'hoursWorkActualSecondJob',
                           'y_salary_m', 'y_salary_m_hu', 'y_ingLab_m', 'y_ingLab_m_ha']

for col in wage_hours_related_vars:

    df_limpio[col] = pd.to_numeric(df_limpio[col], errors='coerce')
    print(f"{col} dtype: {df_limpio[col].dtype}")

# Calculo 'hour_wage' teniendo en cuenta los valores NaN
df_limpio['hour_wage'] = (df_limpio['salario_empleo_principal'].fillna(0) + df_limpio['salario_empleo_secundario'].fillna(0)) / \
                                         ((df_limpio['hoursWorkActualSecondJob'].fillna(0) + df_limpio['hoursWorkUsual'].fillna(0))*4)

# Ajusto para casos donde el denominador es cero (para evitar divisiones por cero)
df_limpio.loc[(df_limpio['hoursWorkActualSecondJob'].fillna(0) + df_limpio['hoursWorkUsual'].fillna(0)) == 0, 'hour_wage'] = np.nan

# Ajusto para casos donde ambos, salario_empleo_principal y salario_empleo_secundario, son NaN
df_limpio.loc[df_limpio['salario_empleo_principal'].isna() & df_limpio['salario_empleo_secundario'].isna(), 'hour_wage'] = np.nan

# Ajusto para casos donde ambos, hoursWorkActualSecondJob y hoursWorkUsual, son NaN
df_limpio.loc[df_limpio['hoursWorkActualSecondJob'].isna() & df_limpio['hoursWorkUsual'].isna(), 'hour_wage'] = np.nan

# Agrego la variable creada a la lista con las variables a comparar
wage_hours_related_vars = wage_hours_related_vars + ['hour_wage']

In [ ]:
# Observo variables de interés
print("Unique values in 'maxEducLevel':")
print(df_limpio['maxEducLevel'].unique())

print("\nUnique values in 'sizeFirm':")
print(df_limpio['sizeFirm'].unique())

print("\nUnique values in 'relab':")
print(df_limpio['relab'].unique())

print("\nUnique values in 'informal':")
print(df_limpio['informal'].unique())

print("\nUnique values in 'sex':")
print(df_limpio['sexo'].unique())

In [ ]:
# Convierto a las que van a ser mis variables independientes a tipo numérico, manejando posibles valores no numéricos (como 'NA')

df_limpio['maxEducLevel'] = pd.to_numeric(df_limpio['maxEducLevel'], errors='coerce')
df_limpio['antig'] = pd.to_numeric(df_limpio['antig'], errors='coerce')
df_limpio['edad'] = pd.to_numeric(df_limpio['edad'], errors='coerce')
df_limpio['sizeFirm'] = pd.to_numeric(df_limpio['sizeFirm'], errors='coerce')
df_limpio['relab'] = pd.to_numeric(df_limpio['relab'], errors='coerce')
df_limpio['informal'] = pd.to_numeric(df_limpio['informal'], errors='coerce')
df_limpio['sexo'] = pd.to_numeric(df_limpio['sexo'], errors='coerce')

In [ ]:
# Genero dummies para variables categóricas
categorical_vars = ["relab", "sizeFirm", "maxEducLevel"]
df_encoded = pd.get_dummies(df_limpio, columns=categorical_vars, drop_first = True)

# Defino un subset con las variables de interés con el format correcto
variables = ["y_ingLab_m_ha", "informal", "antig", "sexo", "edad"] + \
            [col for col in df_encoded.columns if any(var in col for var in categorical_vars)]

subset = df_encoded[variables]

## Descripción de los datos

In [ ]:
# Estadisticos descriptivos para asociadas al salario horario

pd.set_option('display.float_format', '{:,.2f}'.format)

for col in wage_hours_related_vars:

  print("\nEstadísticas descriptivas:")
  print(df_limpio[col].describe())

  zero_count = (df_limpio[col] == 0).sum()
  na_count = df_limpio[col].isna().sum()
  print(f"Number of zeros in '{col}': {zero_count}")
  print(f"Number of missing values (NA) in '{col}': {na_count}")

In [ ]:
import matplotlib.pyplot as plt

# Diccionario con nombres descriptivos para las variables
etiquetas = {
    "salario_empleo_principal": "Salario del empleo principal",
    "salario_empleo_secundario": "Salario del empleo secundario",
    "hoursWorkUsual": "Horas trabajadas",
    "hoursWorkActualSecondJob": "Horas trabajadas en el segundo empleo",
    "y_salary_m": "Salario nominal mensual en empleo principal",
    "y_salary_m_hu": "Salario real horario en empleo principal",
    "y_ingLab_m": "Ingreso laboral nominal mensual",
    "y_ingLab_m_ha": "Ingreso laboral nominal horario",
    "hour_wage": "Salario horario"
}


# Graficar histogramas
for col in wage_hours_related_vars:
    data = df_limpio[col].dropna()

    plt.figure(figsize=(8, 6))  # Tamaño de la figura
    plt.hist(data, bins=30, color='b', edgecolor='black', alpha=0.7)
    plt.title(f"Histograma de {etiquetas.get(col, col)}", fontsize=14)  # Título claro
    plt.xlabel(etiquetas.get(col, col), fontsize=12)
    plt.ylabel("Frecuencia", fontsize=12)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.tight_layout()  # Ajusta los márgenes automáticamente
    plt.show()


In [ ]:

# Graficamos histogramas del logaritmo natural
for col in wage_hours_related_vars:
    data = df_limpio[col].dropna()

    # Excluir valores no positivos para calcular logaritmo natural
    data_log = data[data > 0].apply(np.log)

    plt.figure(figsize=(8, 6))  # Tamaño de la figura
    plt.hist(data_log, bins=30, color='b', edgecolor='black', alpha=0.7)  # Color azul con bordes negros
    plt.title(f"Logaritmo natural de {etiquetas.get(col, col)}", fontsize=14)  # Título claro
    plt.xlabel(f"{etiquetas.get(col, col)} (en logaritmo)", fontsize=12)  # Etiqueta del eje x
    plt.ylabel("Frecuencia", fontsize=12)  # Etiqueta del eje y
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.tight_layout()  # Ajusta los márgenes automáticamente
    plt.show()

In [ ]:
# Calculo y visualizo la matriz de correlación
corr_matrix = subset.corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, cmap="coolwarm", fmt=".2f", cbar=True)
plt.title("Matriz de correlación")
plt.show()

In [ ]:
# Elimino missings

df_limpio = df_limpio[["y_ingLab_m_ha", "edad", "informal", "antig", "relab", "sexo", "sizeFirm","maxEducLevel"]]
df_limpio = df_limpio.dropna()

In [ ]:
# Defino las variables independientes y la dependiente
variables_independientes = ["edad", "informal", "relab", "antig", "sexo", "sizeFirm","maxEducLevel"]

variable_dependiente = "y_ingLab_m_ha"

import matplotlib.pyplot as plt

# Diccionario para renombrar variables con nombres más descriptivos
etiquetas = {
    "edad": "Edad",
    "informal": "Trabajo informal",
    "relab": "Relación laboral",
    "antig": "Antigüedad",
    "sexo": "Sexo",
    "sizeFirm": "Tamaño de la firma",
    "maxEducLevel": "Máximo nivel educativo",
    "y_ingLab_m_ha": "Salario horario"
}

# Creo scatterplots entre la variable dependiente y cada variable independiente
for var in variables_independientes:
    plt.figure(figsize=(8, 6))  # Tamaño de la figura
    plt.scatter(df_limpio[var], df_limpio[variable_dependiente], alpha=0.7)
    plt.title(f"{etiquetas.get(var, var)} vs {etiquetas[variable_dependiente]}", fontsize=14)
    plt.xlabel(etiquetas.get(var, var), fontsize=12)
    plt.ylabel(etiquetas[variable_dependiente], fontsize=12)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.tight_layout()  # Ajusta el diseño para evitar solapamientos
    plt.show()



# Modelos

In [ ]:
# Definimos variables dependiente e independientes, el train set y el test set

X = df_limpio[["edad", "informal", "relab", "antig", "sexo", "sizeFirm","maxEducLevel"]]

y=df_limpio[["y_ingLab_m_ha"]]

X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y,
                                        test_size=0.3,
                                        train_size=0.7,
                                        random_state = 123
                                        )

X_train.head()

,edad,informal,relab,antig,sexo,sizeFirm,maxEducLevel
7741,43,0,1,24,1,5,6.00
22475,32,0,2,144,1,5,7.00
13596,19,1,1,3,0,5,7.00
31707,29,0,1,7,0,5,6.00
15334,19,1,1,1,1,2,6.00


## Modelo 1

In [ ]:
# Definimos el modelo
X0 = np.ones((len(y_train), 1))
model1=  LinearRegression().fit(X0,y_train)
model1.intercept_

array([8872.638145])

In [ ]:
# Media estimada
y_train.mean()

,0
y_ingLab_m_ha,"8,872.64"


In [ ]:
# Predicciones basadas en el modelo 1
X0_test = np.ones((len(y_test), 1))
y_hat_model1 = model1.predict(X0_test)

# Calculamos el Mean Squared Error (MSE) y el Root Mean Squared Error (RMSE)
mse1 = mean_squared_error(y_test, y_hat_model1)
rmse1 = np.sqrt(mse1)

print(f'Root Mean Squared Error: {rmse1}')

## Modelo 2: Intercepto y máximo nivel educativo alcanzado

In [ ]:
# Definimos el modelo
model2=  LinearRegression().fit(X_train[['maxEducLevel']],y_train)
model2.coef_

array([[3026.0805349]])

In [ ]:
# Predicciones basadas en el modelo 2
y_hat_model2 = model2.predict(X_test[['maxEducLevel']])

# Calculamos el Mean Squared Error (MSE) y el Root Mean Squared Error (RMSE)
mse2 = mean_squared_error(y_test, y_hat_model2)
rmse2 = np.sqrt(mse2)

print(f'Root Mean Squared Error: {rmse2}')

Mean Squared Error: 158101108.2962417


## Modelo 3: Intercepto, máximo nivel educativo alcanzado, edad y sexo

In [ ]:
# Definimos el modelo
model3=  LinearRegression().fit(X_train[["edad", "informal","sexo","maxEducLevel"]],y_train)
model3.coef_

array([[  255.91014076, -3152.57752079,  1104.11418988,  3610.61464858]])

In [ ]:
# Predicciones basadas en el modelo 3
y_hat_model3 = model3.predict(X_test[["edad", "informal","sexo","maxEducLevel"]])

# Calculamos el Mean Squared Error (MSE) y el Root Mean Squared Error (RMSE)
mse3 = mean_squared_error(y_test, y_hat_model3)
rmse3 = np.sqrt(mse3)

print(f'Root Mean Squared Error: {rmse3}')

Mean Squared Error: 147855976.75942838


## Modelo 4: Intercepto, máximo nivel educativo alcanzado, edad, sexo, formalidad del empleo y tipo de relación laboral

In [ ]:
# Defino el modelo
model4=  LinearRegression().fit(X_train[["edad", "informal", "relab", "sexo","maxEducLevel"]],y_train)
model4.coef_

array([[  253.5511288 , -3186.20252114,   285.88176382,  1162.31830052,
         3622.75230791]])

In [ ]:
# Predicciones basadas en el modelo 4
y_hat_model4 = model4.predict(X_test[["edad", "informal", "relab", "sexo","maxEducLevel"]])

# Calculamos el Mean Squared Error (MSE) y el Root Mean Squared Error (RMSE)
mse4 = mean_squared_error(y_test, y_hat_model4)
rmse4 = np.sqrt(mse4)

print(f'Root Mean Squared Error: {rmse4}')

Mean Squared Error: 147855617.61562744


## Modelo 5: Intercepto, máximo nivel educativo alcanzado, edad, sexo, formalidad del empleo, tipo de relación laboral, antiguedad, tamaño de la firma y oficio

In [ ]:
# Defino el modelo
model5=  LinearRegression().fit(X_train[["edad", "informal", "relab", "antig", "sexo", "sizeFirm","maxEducLevel"]],y_train)
model5.coef_

array([[ 192.93367159, -510.52004364, 1116.19123836,   20.28176156,
         935.17971178, 1342.49632232, 3160.40275166]])

In [ ]:
# Predicciones basadas en el modelo 5
y_hat_model5 = model5.predict(X_test[["edad", "informal", "relab", "antig", "sexo", "sizeFirm","maxEducLevel"]])

# Calculamos el Mean Squared Error (MSE) y el Root Mean Squared Error (RMSE)
mse5 = mean_squared_error(y_test, y_hat_model5)
rmse5 = np.sqrt(mse5)

print(f'Root Mean Squared Error: {rmse5}')

Mean Squared Error: 144591933.73146513


## Modelo 6: Modelo 3 con interacciones y no linealidades de segundo grado

In [ ]:
poly_2 = PolynomialFeatures(degree=2)
poly_3 = PolynomialFeatures(degree=3)

In [ ]:
# Defino el modelo
X_train_poly_2 = poly_2.fit_transform(X_train[["edad", "informal","sexo","maxEducLevel"]])
model6 =  LinearRegression().fit(X_train_poly_2,y_train)

In [ ]:
# Predicciones basadas en el modelo 6
X_test_poly_6 = poly_2.fit_transform(X_test[["edad", "informal","sexo","maxEducLevel"]])
y_hat_model6 = model6.predict(X_test_poly_6)

# Calculamos el Mean Squared Error (MSE) y el Root Mean Squared Error (RMSE)
mse6 = mean_squared_error(y_test, y_hat_model6)
rmse6 = np.sqrt(mse6)

print(f'Root Mean Squared Error: {rmse6}')

Mean Squared Error: 138959885.52011138


## Modelo 7: Modelo 3 con interacciones y no linealidades de tercer grado

In [ ]:
# Defino el modelo
X_train_poly_3 = poly_3.fit_transform(X_train[["edad", "informal","sexo","maxEducLevel"]])
model7 =  LinearRegression().fit(X_train_poly_3,y_train)

In [ ]:
# Predicciones basadas en el modelo 6
X_test_poly_7 = poly_3.fit_transform(X_test[["edad", "informal","sexo","maxEducLevel"]])
y_hat_model7 = model7.predict(X_test_poly_7)

# Calculamos el Mean Squared Error (MSE) y el Root Mean Squared Error (RMSE)
mse7 = mean_squared_error(y_test, y_hat_model7)
rmse7 = np.sqrt(mse7)

print(f'Root Mean Squared Error: {rmse7}')

Mean Squared Error: 135758679.0348077
